<a href="https://colab.research.google.com/github/AbhinavPeri/OpenAI-Warmup-XOR-Problem/blob/main/OpenAI_Warmup_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI XOR Problem

In [87]:
import numpy as np
import random

# Importing PyTorch Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence, pack_sequence

# Importing PyTorch Lightning Libraries
try:
  import pytorch_lightning as pl
except:
  !pip3 install pytorch_lightning
  import pytorch_lightning as pl

from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

# Loading TensorBoard
%load_ext tensorboard

Global seed set to 1


1

In [88]:
# Custom Dataset For XOR Sequences
class XORSeqDataset(Dataset):
  def __init__(self, size=100000, variable_len=False):
    self.variable_len = variable_len
    self.size = size
    self.data = None
    self.lens = None
    self.generate_data()
  
  def generate_data(self):
    self.data = torch.randint(2, (self.size, 50, 1)).float()
    self.lens = torch.randint(1, 51, (self.size,)) if self.variable_len else torch.ones((self.size,)) * 50
    self.labels = (torch.cumsum(self.data, dim=1) % 2).float().squeeze()
  
  def __len__(self):
    return self.size

  def __getitem__(self, idx):
    return self.data[idx], self.lens[idx], self.labels[idx]


In [89]:
# LSTM Module Used for Classifying a Sequence
class XORParityClassifier(nn.Module):
  def __init__(self, hidden_dim, variable_len=False):
    super().__init__()
    self.variable_len = variable_len

    self.lstm = nn.LSTM(1, hidden_dim, 1, batch_first=True)
    self.classifier = nn.Linear(hidden_dim, 1)
  
  def forward(self, x, x_lens):
    x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
    lstm_output, _ = self.lstm(x_packed)
    output, _ = pad_packed_sequence(lstm_output, batch_first=True)
    output = self.classifier(output).squeeze()
    return output

In [90]:
# Pytorch Lightning Module
class LitXORClassifier(pl.LightningModule):
  def __init__(self, hidden_dim, variable_len=False):
    super().__init__()
    self.dataset = XORSeqDataset(size=10000, variable_len=variable_len)
    self.train_set, self.val_set = random_split(self.dataset, [8000, 2000])
    self.variable_len = variable_len
    self.model = XORParityClassifier(hidden_dim, variable_len=variable_len)
    self.criterion = nn.BCEWithLogitsLoss()
  
  def forward(self, x, x_lens):
    return self.model(x, x_lens)

  def evaluate_model(self, batch):
    x, lens, labels = batch
    lens = lens.cpu()
    output = self.model(x, lens)
    if self.variable_len:
      output_packed = pack_padded_sequence(output, lens, batch_first=True, enforce_sorted=False).data
      labels_packed = pack_padded_sequence(labels, lens, batch_first=True, enforce_sorted=False).data
      loss = self.criterion(output_packed, labels_packed)
      acc = torch.mean(((output_packed > 0.5) == (labels_packed > 0.5)).float())
    else:
      loss = self.criterion(output, labels)
      acc = torch.mean(((output > 0.5) == (labels > 0.5)).float())
    return loss, acc

  def training_step(self, batch, batch_idx):
    loss, acc = self.evaluate_model(batch)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    self.log("train_acc", acc, prog_bar=True, logger=True)
    return loss

  def validation_step(self, batch, batch_idx):
    loss, acc = self.evaluate_model(batch)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    self.log("val_acc", acc, prog_bar=True, logger=True)
    return loss

  def configure_optimizers(self):
    return optim.Adam(self.parameters())
  
  def train_dataloader(self):
    return DataLoader(self.train_set, batch_size=8, num_workers=2)

  def val_dataloader(self):
    return DataLoader(self.val_set, batch_size=8, num_workers=2)



## Training


In [94]:
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=5, stopping_threshold=0.001)
logger1 = TensorBoardLogger("lightning_logs", name="XOR Classifier Without Variable Length")
logger2 = TensorBoardLogger("lightning_logs", name="XOR Classifier With Variable Length")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [95]:
model_static_len = LitXORClassifier(2)
model_var_len = LitXORClassifier(2, variable_len=True)
trainer = pl.Trainer(logger=logger1, callbacks=[early_stopping_callback], gpus=1)
trainer.fit(model_static_len)
trainer = pl.Trainer(logger=logger2, callbacks=[early_stopping_callback], gpus=1)
trainer.fit(model_var_len)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                | Params
--------------------------------------------------
0 | model     | XORParityClassifier | 43    
1 | criterion | BCEWithLogitsLoss   | 0     
--------------------------------------------------
43        Trainable params
0         Non-trainable params
43        Total params
0.000     Total estimated model params size (MB)


Global seed set to 1


## Analysis and Comments

In [ ]:
# Loading TensorBoard
%tensorboard --logdir ./lightning_logs

As you can see, the variable length model has taken longer to converge. This is because the model with fixed size always trains on a sequence of 50 units long while the variable length model trains on sequences that are at most 50 units long. This results in the fixed length model being more exposed to the data, and therefore more penalized to converge to a solution. Initially when I implemented the LSTM solution, I only calculated loss based on the last output of the LSTM. When I tried training it however, it would never converge. Then I realized that by penalizing the model only on the last output, I was allowing it to make mistakes in it's output in the beginning of the sequence. As a result, I could not expect it to perform well on the final output if it did not know how to perform well on the other outputs. This underexposure to the data made it impossible for the model to converge, just like how it is handicapping the variable length model.